In [ ]:
!pip install -e $PWD/awscosts

In [ ]:
# dataframe-related imports
import wikimedia_scraper as ws
from datetime import datetime
import pandas as pd
import numpy as np
from datetime import timedelta

import webish_simulator


# plotting-related imports
import matplotlib.pylab as plt
%matplotlib inline

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure

from bokeh.models import DatetimeTickFormatter, NumeralTickFormatter, BasicTickFormatter
from bokeh.models.tickers import FixedTicker

# Get data from wikipedia source

In [ ]:
project = 'en'

start_date = datetime(2016, 11,  1)
end_date   = datetime(2017, 1, 31)

ws.output_notebook()

traffic_generator = ws.get_traffic_generator(start_date, end_date, projects=(project,))
df = pd.DataFrame(list(traffic_generator))

df.head()

In [ ]:
df = df.set_index(pd.DatetimeIndex(df['date']))
df = df.drop(['date'], axis=1)
df = df.loc[df['project']==project]

df.head()

In [ ]:
month_df = webish_simulator.simulate(df, monthly_scale_factor=50000000)
# month_df = webish_simulator.simulate(df)

month_df.head()

In [ ]:
# plotting avg week

# week_plot = figure(title="wikipedia average week costs", x_axis_type="datetime")

# week_plot.yaxis.formatter = BasicTickFormatter(use_scientific=False)

# week_plot.xaxis.formatter = DatetimeTickFormatter(
#         hours  = [ '%R'    ],
#         days   = [ '%a'    ],
#         months = [ '%a %H' ],
# )

# week_plot.yaxis[0].formatter = NumeralTickFormatter(format='0.000a')

# avg_week_data = week_plot.line(df3.index, df3['lambda_sum'], color="#2222aa", line_width=1)
# avg_week_data = week_plot.line(df3.index, df3['ec2_sum'], color="black", line_width=1)

# output_notebook()
# show(week_plot, notebook_handle=True)
# push_notebook()

plt.plot(month_df['lambda_sum'], color='blue', label='hits')
plt.plot(month_df['ec2_sum'], color='red',label='Original')
